In [3]:
# !pip install Pillow -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip install matplotlib -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip install pandas -i https://pypi.tuna.tsinghua.edu.cn/simple
# !pip install fairseq -i https://pypi.tuna.tsinghua.edu.cn/simple

In [1]:
import os
import sys
import torch
import torch.nn.functional as F
import soundfile as sf
import numpy as np
import librosa
import traceback

from tqdm import tqdm
from fairseq import checkpoint_utils

import sys
sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
from basic import plot_spectrogram

sys.path.append("../")
from pyfiles.utils import trim_audio_and_save

/mntcephfs/lab_data/shoinoue/miniconda3/envs/cuhk/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [2]:
model_path = "/mntcephfs/lab_data/shoinoue/Models/trained_models/hubert/hubert_base_ls960.pt"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("loading model(s) from {}".format(model_path))
models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task(
    [model_path],
    suffix="",
)
model = models[0].half() 
model = model.to(device)
model.eval()

def postprocess(feats, normalize=False):
    if feats.dim() == 2:
        feats = feats.mean(-1)

    assert feats.dim() == 1, feats.dim()

    if normalize:
        with torch.no_grad():
            feats = F.layer_norm(feats, feats.shape)
    return feats

loading model(s) from /mntcephfs/lab_data/shoinoue/Models/trained_models/hubert/hubert_base_ls960.pt


2024-07-26 15:05:59 | INFO | fairseq.tasks.hubert_pretraining | current directory is /home/shoinoue/Git/seq2seq-vc/notebooks
2024-07-26 15:05:59 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': '/checkpoint/wnhsu/data/librispeech/960h/iter/250K_50hz_km100_mp0_65_v2', 'fine_tuning': False, 'labels': ['layer6.km500'], 'label_dir': None, 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
2024-07-26 15:05:59 | INFO | fairseq.models.hubert.hubert | HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activation_dropout': 

In [10]:
### CMU-ARCTIC
# wav_dir = "/mntcephfs/lab_data/shoinoue/Dataset/CMU-ARCTIC/SLT/wav/"
# wav_dir = "/mntcephfs/lab_data/shoinoue/Dataset/PD-AST/SLT/Hindi/wav/"
# output_dir = wav_dir[:-4] + "hubert/"
# add = ""

### LibriTTS-R
wav_dir = "/mntcephfs/lee_dataset/tts/LibriTTS_R/"
output_dir = "/mntcephfs/lab_data/shoinoue/Dataset/LibriTTS_R/features/"
add = "_hubert"


for subdir, dirs, files in tqdm(os.walk(wav_dir)):
    os.makedirs(output_dir + subdir[len(wav_dir):], exist_ok=True)
    for file in files:
# for subdir, dirs, files in os.walk(wav_dir):
#     os.makedirs(output_dir + subdir[len(wav_dir):], exist_ok=True)
#     for file in tqdm(files):
        if file[:2]!="._" and file[-4:]==".wav":
            if subdir[-1]=="/":
                wav_path = subdir + file
            else:
                wav_path = subdir + "/" + file
            savefile = output_dir + wav_path[len(wav_dir):-4] + f"{add}.npy"
            if os.path.exists(savefile):
                continue
            
            tempfile = "temp.wav"
            trim_audio_and_save(wav_path, 16000, tempfile)
            try:
                wav, sr = sf.read(tempfile)
            except Exception as e:
                print("wav load error: ", wav_path)
                traceback.print_exc()
                continue

            if (sr != 16000):
                wav = librosa.resample(wav, orig_sr=sr, target_sr=16000)

            if (wav.shape[0] < 3200 or wav.shape[0] > 30 * 16000):
                print("error : wav too short or too long ", wav_path, wav.shape)
                continue

            feat = torch.from_numpy(wav).float()
            feat = postprocess(feat, normalize=saved_cfg.task.normalize)
            feats = feat.view(1, -1)
            padding_mask = (torch.BoolTensor(feats.shape).fill_(False))

            inputs = {
                "source": feats.half().to(device),
                "padding_mask": padding_mask.to(device),
                "output_layer": 9
            }

            with torch.no_grad():
                logits = model.extract_features(**inputs)

            out_feat = logits[0].squeeze(0)
            out_feat = out_feat.data.detach().cpu().numpy().astype(np.float16)
            np.save(savefile, out_feat) 

109it [00:00, 1086.66it/s]

error : wav too short or too long  /mntcephfs/lee_dataset/tts/LibriTTS_R/test-clean/4507/16021/4507_16021_000025_000000.wav (491139,)
error : wav too short or too long  /mntcephfs/lee_dataset/tts/LibriTTS_R/test-clean/4507/16021/4507_16021_000023_000002.wav (585728,)
error : wav too short or too long  /mntcephfs/lee_dataset/tts/LibriTTS_R/test-clean/7127/75947/7127_75947_000079_000000.wav (3072,)
error : wav too short or too long  /mntcephfs/lee_dataset/tts/LibriTTS_R/test-clean/1089/134686/1089_134686_000008_000000.wav (497152,)


357it [00:01, 221.46it/s] 

error : wav too short or too long  /mntcephfs/lee_dataset/tts/LibriTTS_R/test-clean/7021/79730/7021_79730_000052_000000.wav (517891,)
error : wav too short or too long  /mntcephfs/lee_dataset/tts/LibriTTS_R/train-clean-100/8465/246947/8465_246947_000028_000000.wav (3072,)
error : wav too short or too long  /mntcephfs/lee_dataset/tts/LibriTTS_R/train-clean-100/1743/142912/1743_142912_000015_000001.wav (2560,)


482it [00:01, 329.66it/s]

...

3644it [2:44:13,  5.19s/it]

error : wav too short or too long  /mntcephfs/lee_dataset/tts/LibriTTS_R/train-clean-360/4363/14936/4363_14936_000008_000000.wav (555011,)


3898it [3:00:12,  4.53s/it]

error : wav too short or too long  /mntcephfs/lee_dataset/tts/LibriTTS_R/train-clean-360/14/212/14_212_000011_000004.wav (3072,)
error : wav too short or too long  /mntcephfs/lee_dataset/tts/LibriTTS_R/train-clean-360/14/212/14_212_000018_000001.wav (2560,)
error : wav too short or too long  /mntcephfs/lee_dataset/tts/LibriTTS_R/train-clean-360/14/212/14_212_000011_000009.wav (3072,)


3971it [3:04:58,  2.79s/it]
